<a href="https://colab.research.google.com/github/sanjoy-kumar/Machine-Learning/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Gadiant Decent for Logistic Regression**
-------------------------------------------
**Introduction:**

Gradient Descent is the process of minimizing a function by following the gradients of the cost function. This involves knowing the form of the cost as well as the derivative so that from a given point you know the gradient and can move in that direction, e.g. downhill towards the minimum value.



In [ ]:
#Importing required libraries
import pandas as pd
import numpy as np
import math

In [ ]:
#loading heart.csv file
HeartDF = pd.read_csv('https://raw.githubusercontent.com/tofighi/MachineLearning/master/datasets/heart.csv',index_col=0)
HeartDF.head(10)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
row.names,,,,,,,,,,
1,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1
6,132,6.20,6.47,36.21,Present,62,30.77,14.14,45,0
7,142,4.05,3.38,16.20,Absent,59,20.81,2.62,38,0
8,114,4.08,4.59,14.60,Present,62,23.11,6.72,58,1
9,114,0.00,3.83,19.40,Present,49,24.86,2.49,29,0


Q1. How much is the percentage each class 0 or 1?

In [ ]:
Y = np.array(HeartDF[['chd']])
n = len(Y)
chd_zero = 0
chd_one = 0
garbage =0
for i in range(n):
  if Y[i] == 0:
    chd_zero += 1
  elif Y[i] == 1:
    chd_one += 1
  else:
    garbage += 1

percentage_of_zero = 0
percentage_of_one = 0

percentage_of_zero = (chd_zero/n)*100
percentage_of_one = (chd_one/n)*100
print("Percentage of zero(0) in each class is {:.2f}%.".format(percentage_of_zero))
print("Percentage of one(1) in each class is {:.2f}%.".format(percentage_of_one))



Percentage of zero(0) in each class is 65.37%.
Percentage of one(1) in each class is 34.63%.


# Q2. How many missing values do we have?

In [ ]:
# ----------------Count total NaN at each column in the DataFrame -------------

print("\nCount total NaN(Not a Number) at each column in the DataFrame :\n\n",
      HeartDF.isnull().sum())

#---------------- Count total NaN in a DataFrame-------------------------------

print(" \nCount total NaN(Not a Number) in the DataFrame : \n\n",
       HeartDF.isnull().sum().sum())


Count total NaN(Not a Number) at each column in the DataFrame :

 sbp          0
tobacco      0
ldl          0
adiposity    0
famhist      0
typea        0
obesity      0
alcohol      0
age          0
chd          0
dtype: int64
 
Count total NaN(Not a Number) in the DataFrame : 

 0


Q3. How many categorical variables you have in your features?

Answer: one categorical variable we have in our features. It is '**famhist**'.

Encode the categorical variable:

In [ ]:
HeartDF['famhist']= pd.get_dummies(HeartDF['famhist'])
HeartDF.head(10)

# ----------------Count total NaN at each column in the DataFrame -------------

print("\nCount total NaN(Not a Number) at 'famhist' column after Encoding :\n\n",
      HeartDF['famhist'].isnull().sum())



Count total NaN(Not a Number) at 'famhist' column after Encoding :

 0


Q4. What featues have the maximum corelation?

In [ ]:

print(HeartDF.corr(method ='pearson').abs())
#print(HeartDF.corr().abs().unstack().sort_values(ascending=False))

def get_redundant_pairs(HeartDF):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = HeartDF.columns
    for i in range(0, HeartDF.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(HeartDF, n=5):
    au_corr = HeartDF.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(HeartDF)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("\n---\t --------\t-----------")
print("Top\t Absolute \tCorrelations")
print("---\t --------\t-----------")
print(get_top_abs_correlations(HeartDF, 3))


                sbp   tobacco       ldl  ...   alcohol       age       chd
sbp        1.000000  0.212247  0.158296  ...  0.140096  0.388771  0.192354
tobacco    0.212247  1.000000  0.158905  ...  0.200813  0.450330  0.299718
ldl        0.158296  0.158905  1.000000  ...  0.033403  0.311799  0.263053
adiposity  0.356500  0.286640  0.440432  ...  0.100330  0.625954  0.254121
famhist    0.085645  0.088601  0.161353  ...  0.080520  0.239667  0.272373
typea      0.057454  0.014608  0.044048  ...  0.039498  0.102606  0.103156
obesity    0.238067  0.124529  0.330506  ...  0.051620  0.291777  0.100095
alcohol    0.140096  0.200813  0.033403  ...  1.000000  0.101125  0.062531
age        0.388771  0.450330  0.311799  ...  0.101125  1.000000  0.372973
chd        0.192354  0.299718  0.263053  ...  0.062531  0.372973  1.000000

[10 rows x 10 columns]

---	 --------	-----------
Top	 Absolute 	Correlations
---	 --------	-----------
adiposity  obesity    0.716556
           age        0.625954
tobacco 

**Standardization:**




In [126]:
def mean(HeartDF):
    n = len(HeartDF)
    results = sum(HeartDF)/n
    return results

def standard_deviation(HeartDF):
    

standardized_HeartDF = (HeartDF - mean(HeartDF)) / standard_deviation(HeartDF))


SyntaxError: ignored